# THIS NOTEBOOK IS FOR NEW TESTING DATA <br>
# COMPLETE TESTING USER PROFILE CAN BE READ IN via CSV <br>
# `complete_user_profile_test.csv`

# Creating User Profile

This notebook will look at user habits 

In [2]:
import collections
import pathlib
import random


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import seaborn as sns
sns.set_style('whitegrid')

%matplotlib inline
# To change scientific numbers to float
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [49]:
user_mean = pd.read_csv('test_user_profile.csv')

In [54]:
# need to store user_ids to update vector ids later
user_ids = list(user_mean['user_id'])

# Now we need to turn the product names into numbers

Enter Word2Vec <br>
<a href = 'https://www.youtube.com/watch?v=hQwFeIupNP0'> Word2Vec Explanattion Video </a> <br> 
<a href = 'https://youtu.be/Q2NtCcqmIww'> Word2Vec Coding Video </a> <br>
<a href = 'https://github.com/codebasics/deep-learning-keras-tf-tutorial/blob/master/42_word2vec_gensim/42_word2vec_gensim.ipynb'> Word2Vec Github Link </a> <br><a href = 'https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#sphx-glr-auto-examples-tutorials-run-word2vec-py'> Word2Vec Documentation Link </a>

In [55]:
import gensim

In [56]:
# Remove punctuation and prepare for tokenization 
product_text = user_mean.Products.apply(gensim.utils.simple_preprocess).to_dict()
type(product_text)

dict

In [57]:
# get the number of words in each users product list -> will influence our window size on doc2vec
len_list = []
for key in product_text:
    len_list.append(len(product_text[key]))

In [58]:
average_product_df = pd.DataFrame()

In [59]:
average_product_df['num_words'] = len_list

In [60]:
average_product_df['num_words'].describe() # <- lets use a window size of 660 bc it covers 75% of the data 

count      25.000000
mean     2461.080000
std      1308.464779
min       976.000000
25%      1625.000000
50%      1983.000000
75%      3134.000000
max      6229.000000
Name: num_words, dtype: float64

## Create Vectors
<a href = 'https://www.markhneedham.com/blog/2015/02/12/pythongensim-creating-bigrams-over-how-i-met-your-mother-transcripts/'> NLP Phrases Tutorial </a> <br>


In [61]:
from gensim.models import Phrases
from gensim.models import Word2Vec
import nltk 
import string

In [62]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import common_texts

In [63]:
product_text_dict = product_text ## .to_dict()

## Doc 2 Vec Model 6 - Vector = 10, Window = max <br>
Need to limit the number of vectors so we can reduce the number of features in kmeans

In [65]:
len(product_text_dict)

25

In [66]:
documents = [TaggedDocument(val, [key]) for key, val in product_text_dict.items()]

# Need to figure out window size

model = Doc2Vec(documents, vector_size=10, window=4735, min_count=20, workers=4)

cuser = 24

products = product_text_dict[cuser]
product_document_vector = model.infer_vector(products)
model.dv.most_similar([product_document_vector])

[(6, 0.9277506470680237),
 (4, 0.796838641166687),
 (5, 0.767103374004364),
 (8, 0.7658442854881287),
 (9, 0.7640862464904785),
 (3, 0.7603421211242676),
 (7, 0.7542337775230408),
 (11, 0.7476176619529724),
 (21, 0.7382446527481079),
 (2, 0.7242172360420227)]

# Using Orignial Doc 2 Vec Model 6 <br> 
- limited number of vectors without sacrificing accuracy 
* Next, we add Vectors to dictionary to then add back to `user_mean`

In [67]:
model.save("./final_doc2vec.model")

In [68]:
model.infer_vector(products)

array([0.145596, -4.771446, 0.202195, -9.125681, 2.134288, -7.243256,
       6.717760, 0.357148, 2.048114, 2.091209], dtype=float32)

In [69]:
vector_dict = {}

for key in product_text_dict: 
    product = product_text_dict[key]
    product_document_vector = model.infer_vector(products)
    vector_dict[key] = product_document_vector


In [70]:
# Convert to DataFrame 
vector_df_index = vector_dict.keys()
vector_df = pd.DataFrame.from_dict(vector_dict, orient = 'index')
vector_df.head()

,0,1,2,3,4,5,6,7,8,9
0,-0.408277,-3.820417,0.965710,-7.809166,0.981473,-5.607796,5.943647,-0.436163,2.501771,1.929582
1,-0.344162,-3.912487,1.048274,-8.218671,1.124711,-4.741918,4.620296,-0.752186,2.631187,2.034431
2,0.524475,-4.163799,0.647743,-8.116487,1.595886,-4.618297,4.678394,0.340795,2.556319,2.113552
3,-0.434540,-4.286207,0.701886,-8.103796,1.170677,-5.532918,5.702904,-0.556982,2.569367,1.847137
4,0.048051,-4.848201,0.479462,-8.841644,1.394139,-6.649158,6.462398,-0.362493,2.476308,2.067087


In [71]:
vector_df.reset_index(inplace = True)

In [72]:
vector_df_1 = vector_df
vector_df_1.head()

,index,0,1,2,3,4,5,6,7,8,9
0,0,-0.408277,-3.820417,0.965710,-7.809166,0.981473,-5.607796,5.943647,-0.436163,2.501771,1.929582
1,1,-0.344162,-3.912487,1.048274,-8.218671,1.124711,-4.741918,4.620296,-0.752186,2.631187,2.034431
2,2,0.524475,-4.163799,0.647743,-8.116487,1.595886,-4.618297,4.678394,0.340795,2.556319,2.113552
3,3,-0.434540,-4.286207,0.701886,-8.103796,1.170677,-5.532918,5.702904,-0.556982,2.569367,1.847137
4,4,0.048051,-4.848201,0.479462,-8.841644,1.394139,-6.649158,6.462398,-0.362493,2.476308,2.067087


In [73]:
# need to change index to actual user_id's
vector_df_1['index'] = user_ids

In [75]:
# df.rename(columns={"A": "a", "B": "c"})
user_mean_v = vector_df_1.rename(columns = {'index': 'user_id', 
                             0: 'vector_1', 
                             1: 'vector_2',
                             2: 'vector_3',
                             3: 'vector_4',
                             4: 'vector_5',
                             5: 'vector_6',
                             6: 'vector_7',
                             7: 'vector_8',
                             8: 'vector_9',
                             9: 'vector_10'})
user_mean_v.head()

,user_id,vector_1,vector_2,vector_3,vector_4,vector_5,vector_6,vector_7,vector_8,vector_9,vector_10
0,12345678912345,-0.408277,-3.820417,0.965710,-7.809166,0.981473,-5.607796,5.943647,-0.436163,2.501771,1.929582
1,12345678912346,-0.344162,-3.912487,1.048274,-8.218671,1.124711,-4.741918,4.620296,-0.752186,2.631187,2.034431
2,12345678912347,0.524475,-4.163799,0.647743,-8.116487,1.595886,-4.618297,4.678394,0.340795,2.556319,2.113552
3,12345678912348,-0.434540,-4.286207,0.701886,-8.103796,1.170677,-5.532918,5.702904,-0.556982,2.569367,1.847137
4,12345678912349,0.048051,-4.848201,0.479462,-8.841644,1.394139,-6.649158,6.462398,-0.362493,2.476308,2.067087


### `user_mean_v` is ready, now we need to join `user_mean_v` with `user_mean`

In [76]:
user_mean.head()

,user_id,order_dow,order_hour_of_day,days_since_prior_order,num_orders,num_products,avg_num_products_per_order,Products
0,12345678912345,3.032895,10.707237,5.746711,60,67,1.116667,Bag of Organic Bananas Bag of Organic Bananas ...
1,12345678912346,3.449573,15.948718,6.222222,50,254,5.080000,Bag of Organic Bananas Bag of Organic Bananas ...
2,12345678912347,2.863785,12.504078,4.689233,82,178,2.170732,Romaine Hearts Romaine Hearts Large Alfresco E...
3,12345678912348,3.029491,12.180965,6.297587,62,246,3.967742,Vanilla Milk Chocolate Almond Ice Cream Bars M...
4,12345678912349,1.658098,13.883033,5.143959,48,156,3.250000,Organic Baby Carrots Organic Baby Carrots Orga...


In [77]:
user_mean_join = pd.merge(user_mean, user_mean_v, how = 'left', on = 'user_id')

In [78]:
user_mean_join.drop(columns = 'Products', inplace = True)

In [79]:
user_mean_final = user_mean_join.set_index('user_id')

In [80]:
user_mean_final.head()

,order_dow,order_hour_of_day,days_since_prior_order,num_orders,num_products,avg_num_products_per_order,vector_1,vector_2,vector_3,vector_4,vector_5,vector_6,vector_7,vector_8,vector_9,vector_10
user_id,,,,,,,,,,,,,,,,
12345678912345,3.032895,10.707237,5.746711,60,67,1.116667,-0.408277,-3.820417,0.965710,-7.809166,0.981473,-5.607796,5.943647,-0.436163,2.501771,1.929582
12345678912346,3.449573,15.948718,6.222222,50,254,5.080000,-0.344162,-3.912487,1.048274,-8.218671,1.124711,-4.741918,4.620296,-0.752186,2.631187,2.034431
12345678912347,2.863785,12.504078,4.689233,82,178,2.170732,0.524475,-4.163799,0.647743,-8.116487,1.595886,-4.618297,4.678394,0.340795,2.556319,2.113552
12345678912348,3.029491,12.180965,6.297587,62,246,3.967742,-0.434540,-4.286207,0.701886,-8.103796,1.170677,-5.532918,5.702904,-0.556982,2.569367,1.847137
12345678912349,1.658098,13.883033,5.143959,48,156,3.250000,0.048051,-4.848201,0.479462,-8.841644,1.394139,-6.649158,6.462398,-0.362493,2.476308,2.067087


In [81]:
user_mean_final.to_csv('./complete_user_profile_test.csv')